In [ ]:
#py_wrf_arps version : 2308f94f39bfcb528a6831fa73111bec80e03a5a , the current date is :  2025-02-21 13:59:19.987486
from py_wrf_arps import *
import time
import copy
import scipy
import pandas as pd
%matplotlib inline

In [ ]:
sim = Proj("../../../../", "14_20200515/", ["05"], 
        ["WRF"], 
        tab_test=["30"])
dom = sim.get_dom("05")
SBZvec = np.linspace(0.06, 2.4, 40)

## init p, interp

In [ ]:
p = {}
# itime = ("2020-05-17-12", "2020-05-17-16", "1h")
itime = ("2020-05-17-12-40", "2020-05-17-14-30")
kw_get = {
    "itime" : itime,
    "crop" : ([0, 40], [35, 250], [15, 210]),
    "saved" : p,
}

In [ ]:
def interp_TKE_SBZ(gr):
    gr["SBZ2"] = SBZvec
    SBZ = gr["SBZ"].values
    varnames_temp = copy.copy(varnames)
    for v in varnames :  
        if v in ["GWIL", "GWIR", "GWM2D", "X", "Y", "WD180", "MH", "GWMASK4", "GWMASK5", "RI"] : 
            varnames_temp.remove(v)
    for v in varnames_temp :
        gr[v] = np.interp(SBZvec, SBZ, gr[v], left = np.nan, right=np.nan)
    # suprisingly slower version
    # f = scipy.interpolate.interp1d(gr["SBZ"], gr[varnames_temp], axis=0, bounds_error = False)
    # gr[varnames_temp] = f(SBZvec)
    return gr


## df : skip if read_pickle

In [ ]:
TIME = sim.get_data("05", "TIME", **kw_get)

In [ ]:
if "GWM4D" in p : del p["GWM4D"]
varnames = ["GWMASK4", "GWMASK5", "GWIL", "Z", "COASTDIST","WD180", "GWM4D",
            "GWR", "PTV_C", "TKE_SFS", "U", "V", "W", "X", "Y","SHEAR", "NBV2", 
            "GWCM2U", "GWCM2V", "GWCM2W", "GWCTKE", "GWITKE", 
            "M2U", "M2V", "M2W", "COVUV", "COVVW", "COVUW", "TKE", 
            "GWM4_M2U", "GWM4_M2V", "GWM4_COVUV", "GWM4_COVVW", "GWM4_COVUW",
            "DXW_U", "DYW_U", "DZ_U", "DXW_V", "DYW_V", "DZ_V", "DXW_W", "DYW_W", "DZ_W",
            "PUU1", "PUU2", "PUU3", "PVV1", "PVV2", "PVV3", "PWW1", "PWW2", "PWW3", "PUU", "PVV", "PWW",
            "GWM4_PUU1", "GWM4_PUU2", "GWM4_PUU3", "GWM4_PVV1", "GWM4_PVV2", "GWM4_PVV3", "GWM4_PWW1", "GWM4_PWW2", "GWM4_PWW3", "GWM4_PUU", "GWM4_PVV", "GWM4_PWW",
            "GWM4_U", "GWM4_V",
            
           ]
_ = sim.get_data("05", varnames, **kw_get)
TIME = sim.get_data("05", "TIME", **kw_get)
p2 = {}
NT, NZ, NY, NX = p["Z"].shape
p["it"] = np.arange(len(TIME))
p["iz"] = np.arange(NZ)
# p["GWLAM"] = GWLAM
# p["GWS"] = GWS
# for v in ["it", "GWLAM", "GWS"] :
for v in ["it"] :
    p[v] = np.expand_dims(p[v], axis=(1, 2, 3))
for v in ["it", "GWM4D"] :
    p[v] = np.repeat(p[v], NZ, axis=1)
    p[v] = np.repeat(p[v], NY, axis=2)
    p[v] = np.repeat(p[v], NX, axis=3)
for v in ["iz"] :
    p[v] = np.expand_dims(p[v], axis=(0, 2, 3))
    p[v] = np.repeat(p[v], NT, axis=0)
    p[v] = np.repeat(p[v], NY, axis=2)
    p[v] = np.repeat(p[v], NX, axis=3)

In [ ]:
for v in varnames :
    if p[v].ndim == 3 :
        p[v] = np.expand_dims(p[v], axis=1)
        p[v] = np.repeat(p[v], NZ, axis=1)
    if p[v].ndim == 4 :
        if p[v].shape != (NT, NZ, NY, NX) :
            for idim, dim in enumerate([NT, NZ, NY, NX]) :
                if p[v].shape[idim] == 1 :
                    p[v] = np.repeat(p[v], dim, axis=idim)           
        p2[v] = p[v].flatten()
p2["it"] = p["it"].flatten()
p2["iz"] = p["iz"].flatten()

In [ ]:
df0 = pd.DataFrame(p2)

In [ ]:
df = df0[df0["GWMASK4"]==1]
df["ir*"] = pd.cut(df["GWR"], bins=np.linspace(0, 1, 9), labels=False)
df = df.rename(columns={'GWIL': 'il'})
df = df.set_index(["it", "iz", "il", "ir*", "X", "Y"])
df["TIME"] = manage_time.to_hour(TIME[list(df.index.get_level_values('it'))])

In [ ]:
df5 = df0[df0["GWMASK5"]==1]
df5 = df5.rename(columns={'GWIL': 'il'})
df5 = df5.set_index(["it", "iz", "il"])
df5["TIME"] = manage_time.to_hour(TIME[list(df5.index.get_level_values('it'))])
df5 = df5.groupby(["it", "iz", "il"]).mean()

In [ ]:
itvec = df.index.get_level_values("it").unique()
izvec = df.index.get_level_values("iz").unique()
ilvec = df.index.get_level_values("il").unique()
irvec = df.index.get_level_values("ir*").unique()
NT = len(df.index.get_level_values("it").unique())
NZ = len(df.index.get_level_values("iz").unique())
NL = len(df.index.get_level_values("il").unique())
NR = len(df.index.get_level_values("ir*").unique())

## Trouver z0 et zc pour chaque profil : skip if read_pickle

In [ ]:
def calculate_Tm_Zm(gr): # Pour chaque block, à chaque t, à chaque z, moyenne de Z, U, V, T, WD
    gr["Tm"] = np.mean(gr["PTV_C"])
    gr["Zm"] = np.mean(gr["Z"])
    gr["Um"] = np.mean(gr["U"])
    gr["Vm"] = np.mean(gr["V"])
    gr["WD180m"] = manage_angle.angle180(manage_angle.UV2WD_deg(np.array(gr["Um"]), np.array(gr["Vm"])))
    return gr

def calculate_SBZ(gr):
    _, _, s = manage_list.longest_increasing_sequence(np.array(gr["Tm"]))
    _, _, s2 = manage_list.longest_increasing_sequence(np.array(gr["PTV_C"]))
    gr["GWZ0"] = gr["Z"]*np.nan
    gr["GWZ0"][s2] = np.interp(gr["PTV_C"][s2], gr["Tm"][s], gr["Zm"][s], left = np.nan, right=np.nan)
    pos = np.isnan(gr["GWZ0"])
    if np.any(pos) :
        gr["GWZ0"][pos] = gr["Z"][pos]
    gr["SBZC"] = np.interp(-45, gr["WD180"], gr["GWZ0"]) # Pareil mais pour chaque profil
    gr["SBZ"] = gr["GWZ0"]/gr["SBZC"]
    return gr


In [ ]:
def calculate_SBZ5(gr):
    gr["SBZC"] = np.interp(-45, gr["WD180"], gr["Z"]) # La hauteur caractéristique est définie avec WD = 45°
    gr["SBZ"] = gr["Z"]/gr["SBZC"]
    return gr

In [ ]:
df = df.groupby(['it', 'il', 'iz'], group_keys=False).apply(calculate_Tm_Zm)
df = df.groupby(['it', 'X', 'Y'], group_keys=False).apply(calculate_SBZ)

In [ ]:
df5 = df5.groupby(['it', 'il'], group_keys=False).apply(calculate_SBZ5)

In [ ]:
dg = df.groupby(['it', 'X', 'Y'], group_keys=False).apply(interp_TKE_SBZ)

In [ ]:
dg5 = df5.groupby(['it', 'il'], group_keys=False).apply(interp_TKE_SBZ)

In [ ]:
def calculate_after_interp(gr)
    gr["MH"] = np.sqrt(gr["U"]**2 + gr["V"]**2)
    gr["WD180"] = manage_angle.angle180(np.array(manage_angle.UV2WD_deg(gr["U"], gr["V"])))
    # gr["RI"] = gr["NBV2"]/gr["SHEAR"]**2
    # gr["M2UV"] = gr["M2U"] + gr["M2V"]
    return gr
df = calculate_after_interp(df)
df5 = calculate_after_interp(df5)

In [ ]:
(,)

## save : skip if read_pickle

In [ ]:
df.to_pickle(sim.postprocdir+"GW_M4_df_total5")

In [ ]:
dg.to_pickle(sim.postprocdir+"GW_M4_dg_total5")

In [ ]:
df5.to_pickle(sim.postprocdir+"GW_M5_df_total5")

In [ ]:
dg5.to_pickle(sim.postprocdir+"GW_M5_dg_total5")

## get

In [ ]:
df = pd.read_pickle(sim.postprocdir+"GW_M4_df_total5")
df5 = pd.read_pickle(sim.postprocdir+"GW_M5_df_total5")

In [ ]:
dg = pd.read_pickle(sim.postprocdir+"GW_M4_dg_total5")
dg5 = pd.read_pickle(sim.postprocdir+"GW_M5_dg_total5")

## append

In [ ]:
def append_vars(df, df5, varnames, kw_get, p):
    varnames = varnames + ["GWR", "GWIL", "it", "iz", "X", "Y", "GWMASK4", "GWMASK5", "Z"]
    _ = sim.get_data("05", varnames, **kw_get, print_level=2)
    NT, NZ, NY, NX = p["Z"].shape
    p["it"] = np.expand_dims(np.arange(NT), axis=(1, 2, 3))
    p2bis = {}
    for v in varnames :
        if p[v].ndim == 3 :
            p[v] = np.expand_dims(p[v], axis=1)
            p[v] = np.repeat(p[v], NZ, axis=1)
        if p[v].ndim == 4 :
            if p[v].shape != (NT, NZ, NY, NX) :
                for idim, dim in enumerate([NT, NZ, NY, NX]) :
                    if p[v].shape[idim] == 1 :
                        p[v] = np.repeat(p[v], dim, axis=idim)           
            p2bis[v] = p[v].flatten()
    df0bis = pd.DataFrame(p2bis)
    
    dfbis = df0bis[df0bis["GWMASK4"]==1]
    dfbis["ir*"] = pd.cut(dfbis["GWR"], bins=np.linspace(0, 1, 9), labels=False)
    dfbis = dfbis.rename(columns={'GWIL': 'il'})
    dfbis = dfbis.set_index(["it", "iz", "il", "ir*", "X", "Y"])
    dfbis = dfbis.drop(["GWR", "GWMASK4", "GWMASK5", "Z"], axis=1)
    dfnew = pd.concat([df, dfbis], axis=1)
    
    df5bis = df0bis[df0bis["GWMASK5"]==1]
    df5bis = df5bis.rename(columns={'GWIL': 'il'})
    df5bis = df5bis.set_index(["it", "iz", "il"])
    df5bis = df5bis.groupby(["it", "iz", "il"]).mean()
    df5bis = df5bis.drop(["X", "Y", "GWR", "GWMASK4", "GWMASK5", "Z"], axis=1)
    df5new = pd.concat([df5, df5bis], axis=1)
    
    return dfnew, df5new

In [ ]:
varnames = ["GWM4_COVVP"]
df, df5 = append_vars(df, df5, varnames, kw_get, p)

In [ ]:
dg5new = df5.groupby(['it', 'X', 'Y'], group_keys=False).apply(interp_TKE_SBZ)

In [ ]:
dgnew = df.groupby(['it', 'X', 'Y'], group_keys=False).apply(interp_TKE_SBZ)

In [ ]:
for v in varnames :
    dg5[v] = dg5new[v]
    dg[v] = dgnew[v]

## TKE res / sfs

In [ ]:
import matplotlib
c = list(matplotlib.rcParams["axes.prop_cycle"])
# for it in range(0, 19, 3) :
fig = plt.figure(figsize = [6, 8], dpi = 120, layout="tight")
ax = plt.subplot(111)
TIME = df5["TIME"].unique()
for i_it, it in enumerate([2, 8]) :
# for it in [6] :
    dfi = df.loc[it, :, :, :].groupby(["iz"]).mean()
    Z = dfi["Z"]
    TKEtot = dfi["TKE"]
    TKEGW = dfi["GWITKE"]
    TKECGW = dfi["GWCTKE"]
    TKE_SFS = dfi["TKE_SFS"]
    
    plt.plot(TKE_SFS, Z, "--", color=manage_plot.colors[i_it+2])
    plt.plot(TKECGW, Z, "-.", color=manage_plot.colors[i_it+2])
    plt.plot(TKEtot, Z, "-", color=manage_plot.colors[i_it+2], label=str(round(TIME[it])) + "00 UTC")
    

plt.plot(TKE_SFS-1, Z, "--", color="k", label="$k_{sgs}$")
plt.plot(TKECGW-1, Z, "-.", color="k", label="$k$")
plt.plot(TKEtot-1, Z, "-", color="k", label="$k_{tot}$")
plt.xlim([0, 0.45])
plt.ylim([0, 800])
# plt.title(TIME[it])
plt.xlabel("TKE ($m^2.s^{-2}$)")
plt.ylabel("Height ($m$)")
ax.set_xticklabels([0, 0.1, 0.2, 0.3, 0.4])
plt.grid()
plt.legend(loc="upper right")
# fig.savefig(sim.figdir+"t30/art1/profile_TKE_res_GW_1300_1400", dpi = 120, layout="tight")

## TKE peak height

In [ ]:
def find_TKE_peak(group) :
    Z = np.array(group["Z"])
    SBZ = np.array(group["SBZ"])
    TKE = np.array(group["TKE"])
    im = scipy.signal.find_peaks(TKE, prominence=0.03)
    im = list(im[0])
    # mask = np.logical_and(Z[im] > 0, Z[im]< 400)
    mask = np.logical_and(SBZ[im] > 0.2, SBZ[im]< 2)
    im = np.array(im)[mask]
    if len(im) > 0 :
        i = im[np.nanargmax(TKE[im])]
        Zi = Z[i]
        SBZi = SBZ[i]
    else :
        i = -1
        Zi = np.nan
        SBZi = np.nan
    group["IP"] = i
    group["ZP"] = Zi
    group["SBZP"] = SBZi
    return group

In [ ]:
def calculate_ZP_var(group, varname):
    if type(varname) in [str] :
        group[varname+"_ZP"] = np.interp(group["ZP"], group["Z"], group[varname])
    elif type(varname) in [list] :
        for v in varname :
            group[v+"_ZP"] = np.interp(group["ZP"], group["Z"], group[v])
    return group

In [ ]:
df5 = df5.groupby(['it', 'il'], group_keys=False).apply(find_TKE_peak)
df5 = df5.groupby(['it', 'il'], group_keys=False).apply(calculate_ZP_var, ["TKE"])

In [ ]:
dfi = df5.loc[:, 0, :]
CDIlim = [0, 4, 8, 12, 60]
params = []

dz_random = 10

params = [{"same_ax" : True,
    "X" : dfi["SBZC"]+(np.random.rand(len(dfi["TIME"]))-0.5)*dz_random, "Y" : dfi["ZP"]+(np.random.rand(len(dfi["TIME"]))-0.5)*dz_random, 
    "Z" : -dfi["COASTDIST"]/1000, "cmap" : "Spectral", "clim" : [0, 16], "discrete" : 4,
    "kwargs_plt" : {"s" : 100*dfi["TKE_ZP"], "zorder" : 3}, 
    "xlim" : [0, 300], "ylim" : [0, 300],
    "xlabel" : "$z_c$ (m)", "ylabel" : "$k_{tot}$ peak height (m)", "clabel" : "Distance from coastline (km)",
    "savepath" : "t30/art1/M5_ZP_SBZC", "DX_subplots" : 10,
    "dpi" : 120,
},{
    "same_ax" : True, "X" : [0, 300], "Y" : [0, 300], "style" : "k",
}]

params.append({
    "same_ax" : True, "X" : [-100], "Y" : [0], "Z" : [0],
    "label" : "$0.1\ m^2.s^{-2}$", "kwargs_plt" : {"s" : 10}, "cmap" : "copper","plot_cbar" : False,
})
params.append({
    "same" : -1, "label" : "$0.3\ m^2.s^{-2}$", "kwargs_plt" : {"s" : 30}
})
fig = sim.plot_fig(params)

## Profil instantanné

In [ ]:
its = [2, 8]
ils = [[17, 23], [15, 22]]
params = []
for i_it, it in enumerate(its) :
    print(df5.loc[it]["TIME"].unique())
    for i_il, il in enumerate(ils[i_it]) :
        dfitl = df5.loc[it, :, il]
        print(round(-np.mean(dfitl["COASTDIST"])/1000))
        params.append({
            "X" : dfitl["TKE"], "Y" : dfitl["Z"], "same_ax" : i_il > 0, 
            "label" : f"{round(-np.mean(dfitl['COASTDIST'])/1000)} km",
            "savepath" : "t30/art1/M5_profiles_TKE_Z_instant2",
            "DX_subplots" : 4, "xlim" : [0, 0.3], "ylim" : [0, 800],
            "kwargs_plt" : {
                "linewidth" : 2,
                "color" : manage_plot.colors[2*i_it + i_il],
            },
            "dpi" : 120,
            "ylabel" : "Height (m)" if i_it < 1 else None, 
            "xlabel" : "$k_{tot}$ (m$^2$.s$^{-2}$)",
            "legend_loc" : "upper right",
            "xticks" : [0.0, 0.1, 0.2],  
            "yticklabels" : [] if i_it > 0 else None,
        })
        params.append({
            "typ" : "AXHLINE", "Y" : np.mean(dfitl["SBZC"]), "kwargs_plt":  {"color" : manage_plot.colors[2*i_it + i_il], "linewidth" : 2, "linestyle" : "--"}
        })
        
params.append({
    "X" : [-1, -0.5], "Y" : [0, 1], "kwargs_plt":  {"color" : "k", "linewidth" : 2, "linestyle" : "--"}, "label" : "$z_c$", "same_ax" : True,
    "legend_loc" : "upper right",
})
fig = sim.plot_fig(params)
    

## Production shear

In [ ]:
st2 = slice(6, 12)
dg["ir*2"] = pd.cut(df["GWR"], bins=np.linspace(0, 1, 25), labels=False)
dg2D = dg.loc[st2, :, :, :].groupby(["iz", "ir*2"]).mean().unstack()
# dg2D = dg.loc[st2, :, :, :].fillna(np.inf).groupby(["iz", "ir*2"]).mean().replace(np.inf, np.nan).unstack()

n = -3
zstr = "SBZ2"
if zstr == "SBZ2" : ylabel = "$z^*$"
else : ylabel = "Height (m)"
params = [{
    "typ" : "2D", "X" : np.array(dg2D["GWR"]), "Y" : np.array(dg2D[zstr]), "Z" : np.array(dg2D["TKE"]), "cmap" : 0,# "xlabel" : "$\phi_W/2\pi$", 
    "ylabel" : ylabel, "clabel" : "$<k_{tot}>_{st}$ (m$^2$.s$^{-2}$)", "xlim" : [0, 1], "xticks" : np.arange(0, 1.1, 0.125), "xticklabels" : ["T","|","A","|","C","|","D","|", "T"],
    "same_fig" : False, "savepath" : f"t30/GW/06_2DV_{zstr}_TKE", #"dpi" : 120,
},{ "typ" : "QUIVER", "U" : np.array(dg2D["GWM4_U"]), "V" : 1800*np.array(dg2D["W"])*(1*zstr=="Z"), "kwargs_plt" : {"color" : "k", "scale_units" : "xy", "angles" : "xy"}, "stepx" : 12,
},{ "typ" : "CONTOUR", "Z" : np.array(dg2D["GWCTKE"]), "kwargs_plt" : {"levels" : np.arange(0, 0.35, 0.05)}, #"clabel" : False,
},{ "same" : n, "Z" : dg2D["GWCTKE"],  "clabel" : "$<k>_{st}$ (m$^2$.s$^{-2}$)",
   "savepath" : f"t30/GW/06_2DV_{zstr}_GWCTKE",
},{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["DZ_GWM4_V"],  "clabel" : "$\\frac{\\partial V_r}{\\partial Z}\ (s^{-1})$", "cmap" : 0, "clim" : "centered",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_DZ_GWM4_V",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_M2V"],  "clabel" : "$<v_r'^2>$ (m$^2$.s$^{-2}$)", "cmap" : 0, "clim" : None,
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_M2V",
# },{"same" : n,},{"same" : n,
   
# },{ "same" : n, "Z" : dg2D["GWM4_COVVW"],  "clabel" : "$<v_r'w'>$ (m$^2$.s$^{-2}$)", "cmap" : "bwr", "clim" : "centered",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_COVVW",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["GWM4_TVW"])*1e4, "cmap" : "bwr", "clim" : [-11, 11], "discrete" : 11,
#    "savepath" : f"t30/GW/06_2DV_{zstr}_TVW", "clabel" : "$<v_r'w'>$ pressure transport (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(-dg2D["GWM4_DYW_COVWP"]/dg2D["RHO"])*1e4,
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_DYW_COVWP", "clabel" : "GWM4_DYW_COVWP (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(-dg2D["DZ_GWM4_COVVP"]/dg2D["RHO"])*1e4,
#    "savepath" : f"t30/GW/06_2DV_{zstr}_DZ_GWM4_COVVP", "clabel" : "DZ_GWM4_COVVP (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["GWM4_PVW"])*1e4, "clabel" : "$<v_r'w'>$ shear production (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PVW",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["DTW_GWM4_COVVW"])*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_DTW_GWM4_COVVW", "cmap" : "bwr", "clabel" : "$<v_r'w'>_{,t}$ (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["GWM4_AVW"])*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_AVW", "cmap" : "bwr", "clabel" : "$<v_r'w'>$ advection (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["DTW_GWM4_COVVW"] - dg2D["GWM4_AVW"] - dg2D["GWM4_TVW"] - dg2D["GWM4_PVW"])*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_OVW", "cmap" : "bwr", "clabel" : "$<v_r'w'>$ residual (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
   
# },{ "same" : n, "Z" : dg2D["GWM4_PVV"]*1e4, "cmap" : "bwr", "clim" : [-11, 11], "discrete" : 11,  "clabel" : "PRVV (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PVV",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_AVV"]*1e4,   "clabel" : "ARVV (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_AVV",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["GWM4_PVV1"])*1e4, "cmap" : "bwr", "clim" : [-11, 11], "discrete" : 11,  "clabel" : "$-<v_r'w'> \\frac{\\partial V_r}{\\partial Z}$ (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PVV",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["GWM4_PUU1"])*1e4, "cmap" : "bwr", "clim" : [-11, 11], "discrete" : 11,  "clabel" : "$-<u_r'^2> \\frac{\\partial U_r}{\\partial X_r}$ (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PVV",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_PVV"]*1e4-dg2D["GWM4_AVV"]*1e4, "clabel" : "PRVV-ARVV (10$^{-4}$ m$^2$.s$^{-3}$)",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PVV-AVV",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_U"], "cmap" : 0, "clim" : None, "discrete" : None, "clabel" : "$U_r\ (m.s^{-1})$",
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_U",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_V"], 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_V", "clabel" : "$V_r\ (m.s^{-1})$",
# },{"same" : n,},{"same" : n,
},{ "same" : n, "Z" : np.array(dg2D["GWM4_PTKE"])*1e4, 
   "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PTKE", "cmap" : "bwr", "clim" :  [-5.5, 5.5],  "clabel" : "Shear production (10$^{-4}$ m$^2$.s$^{-3}$)", "discrete" : 11,
},{"same" : n,},{"same" : n,
},{ "same" : n, "Z" : np.array(dg2D["GWM4_ATKE"])*1e4, 
   "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_ATKE", "clabel" : "Advection (10$^{-4}$ m$^2$.s$^{-3}$)",
},{"same" : n,},{"same" : n,
},{ "same" : n, "Z" : np.array(dg2D["TTKE"])*1e4, 
   "savepath" : f"t30/GW/06_2DV_{zstr}_TTKE", "clabel" : "Pressure transport (10$^{-4}$ m$^2$.s$^{-3}$)",
},{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_PTKE"]*1e4-dg2D["GWM4_ATKE"]*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_PTKE-ATKE", "clabel" : "PTKE-ATKE (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
},{ "same" : n, "Z" : np.array(dg2D["DTW_TKE"])*1e4, 
   "savepath" : f"t30/GW/06_2DV_{zstr}_DTW_TKE", "cmap" : "bwr", "clim" :  [-5.5, 5.5], "discrete" : 11, "clabel" : "Variation rate (10$^{-4}$ m$^2$.s$^{-3}$)",
},{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : np.array(dg2D["DTW_TKE"]-dg2D["GWM4_ATKE"]-dg2D["GWM4_PTKE"])*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_O", "cmap" : "bwr", "clim" :  [-5.5, 5.5], "discrete" : 11, "clabel" : "$O$ (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
},{ "same" : n, "Z" : np.array(dg2D["DTW_TKE"]-dg2D["GWM4_ATKE"]-dg2D["GWM4_PTKE"]-dg2D["TTKE"])*1e4, 
   "savepath" : f"t30/GW/06_2DV_{zstr}_O2", "cmap" : "bwr", "clim" :  [-5.5, 5.5], "discrete" : 11, "clabel" : "$<O>_{st}$ (10$^{-4}$ m$^2$.s$^{-3}$)",
},{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_COVVW"], 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_COVVW", "cmap" : "bwr", "clim" :  "centered", "discrete" : None, "clabel" : "$<v_r'w'>\ (m$^2$.s$^{-2}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["M2W"], 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_M2W", "cmap" : 0, "clim" :  None, "discrete" : None, "clabel" : "$<w'^2>\ (m$^2$.s$^{-2}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["DTW_GWM4_COVVW"]*1e4, 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_DTW_GWM4_COVVW", "cmap" : "bwr", "clim" :  [-5.5, 5.5], "discrete" : 11, "clabel" : "$<v_r'w'>_{,t}$ (10$^{-4}$ m$^2$.s$^{-3}$)",
# },{"same" : n,},{"same" : n,
# },{ "same" : n, "Z" : dg2D["GWM4_COVUV"], 
#    "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_COVUV", "cmap" : "bwr", "clim" :  "centered", "discrete" : None, "clabel" : "$<u_r'v_r'>\ (m$^2$.s$^{-2}$)",
# },{"same" : n,},{"same" : n,
}]
# for v in ["PUU1", "PUU2", "PUU3", "PVV1", "PVV2", "PVV3", "PWW1", "PWW2", "PWW3"]:
# # for v in ["PVW1", "PVW2", "PVW3", "PVW4", "PVW5", "PVW6", "PVW", "AVW1", "AVW2", "AVW3", "AVW"]:
#     params.append({ "same" : n, "Z" : dg2D[f"GWM4_{v}"]*1e4, "clabel" : "GWM4_"+v+" (10$^{-4}$ m$^2$.s$^{-3}$)", "cmap" : "bwr", "clim" : [-11, 11], "discrete" : 11,
#        "savepath" : f"t30/GW/06_2DV_{zstr}_GWM4_{v}", 
#     })
#     params.append({"same" : n,})
#     params.append({"same" : n,})
fig = sim.plot_fig(params)

## Profile k as a function of z* and phi

In [ ]:
zindex = "SBZ2"
varname = "GWCTKE"
varZ = "SBZ2"
saved_varnames = ["GWCTKE", "SBZ2", "GWR"]
dfis = dg.fillna(np.inf).groupby([zindex, "ir*"])[saved_varnames].mean().replace(np.inf, np.nan)
dfi5 = dg5.fillna(np.inf).groupby([zindex])[saved_varnames].mean().replace(np.inf, np.nan)

In [ ]:
from matplotlib.colors import Normalize
params = [{
    "X" : dfi5[varname], "Y" : dfi5[varZ],
    "kwargs_plt" : {"color" : "k", "linewidth" : 3},
    "label" : "Outside the GW",
}]

NR = 8
cmap_name = "tab10"
cmap = plt.get_cmap(cmap_name, NR)  # Choix de la colormap
norm = Normalize(vmin=0, vmax=1)  # Normalisation des valeurs Z
phase0 = np.linspace(0, 1, NR+1)
for ir in range(NR) :
    dfi = dfis.xs(ir, level='ir*')
    color = cmap(norm(np.mean(dfi["GWR"])))
    params.append({
        "X" : dfi[varname], "Y" : dfi[varZ], "same_ax" : True,
        "kwargs_plt" : {"color" : color,}, #"label" : round(np.mean(dfi["GWR"]), 2),
        "savepath" : "t30/GW/08_profile_GWCTKE_SBZ2_M45",
        "xlim" : [0, 0.25], "ylim" : [0, 2],
        "xlabel" : "$<k>_{st}$ (m$^2$.s$^{-2}$)",
        "ylabel" : "$z^*$",
        "dpi" : 120,
        "xticks" : np.arange(0, 0.26, 0.05),
    })
    params.append({
        "X" : dfi[varname], "Y" : dfi[varZ], "Z" : dfi["GWR"], "same_ax" : True,
        "cmap" : cmap_name, "clim" : [0, 1],"plot_cbar" : ir==0, "discrete" : 8,
        "dpi" : 120,
        # "clabel" : "Phase $\phi_W/2\pi$",
        "ticklabels" : ["T","$-$","A","$-$","C","$-$","D","$-$", "T"],
    })
    phasei = np.linspace(phase0[ir], phase0[ir+1], 10)
    # ampi = np.sin(2*np.pi*(phasei-0.25))
    ampi = np.sin(2*np.pi*(phasei-0.25))
    params.append({
        "X" : 0.125+0.08*phasei, "Y" : 0.3+0.2*ampi, "same_ax" : True,
        "dpi" : 120,
        "kwargs_plt" : {"color" : color, "linewidth" : 5}, #"label" : round(np.mean(dfi["GWR"]), 2),
    })
    params.append({
        "typ" : "TEXT", "x" : 0.121+0.08*phasei[0], "y" : 0.27+0.2*ampi[0], "Z" : ["T","","A","","C","","D","", "T"][ir]
    })
    
params.append({
    "typ" : "TEXT", "x" : 0.121+0.08*phase0[-1], "y" : 0.27+0.2*ampi[-1], "Z" : "T",
})

fig = sim.plot_fig(params)
